Import Libraries

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def calculate_ema(data, window_size):
    weights = np.exp(np.linspace(-1.0, 0.0, window_size))
    weights /= weights.sum()
    ema = np.convolve(data, weights, mode='full')[:len(data)]
    ema[:window_size - 1] = ema[window_size - 1]
    return ema

def calculate_std(data, window_size):
    return np.std(data - pd.Series(data).rolling(window=window_size).mean(), ddof=0)


Generating Signals


In [ ]:
def generate_signals(price_data, ema_window, std_window):
    ema = calculate_ema(price_data, ema_window)
    std = calculate_std(price_data, std_window)

    signals = np.zeros_like(price_data)
    signals[price_data > ema + 2 * std] = -1  # Sell signal
    signals[price_data < ema - 2 * std] = 1   # Buy signal

    return signals

In [ ]:
def build_and_train_model(X_train, y_train):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(10, input_shape=(X_train.shape[1],), activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, batch_size=32)

    return model


Risk Management Function for portfolio

In [ ]:
def apply_risk_management(positions, price_data, risk_percent=0.02):
    capital = 100000  # Initial capital
    position_size = capital * risk_percent

    current_position = 0
    entry_price = 0

    equity_curve = []

    for i in range(len(positions)):
        if positions[i] == 1 and current_position == 0:  # Buy signal
            entry_price = price_data[i]
            current_position = position_size / entry_price
            equity_curve.append(capital)
        elif positions[i] == -1 and current_position > 0:  # Sell signal
            exit_price = price_data[i]
            capital = current_position * exit_price
            current_position = 0
            equity_curve.append(capital)
        else:
            equity_curve.append(capital)

    return np.array(equity_curve)


In [ ]:
# Read training data from CSV file
training_data = pd.read_csv('training_data.csv')
training_price_data = training_data['Close'].values


In [ ]:
# Use signals to train the model
ema_window_size = 20
std_window_size = 20

training_signals = generate_signals(training_price_data, ema_window_size, std_window_size)

X_train = np.column_stack([training_price_data, calculate_ema(training_price_data, ema_window_size)])
y_train = np.where(training_signals == 1, 1, 0)

model = build_and_train_model(X_train, y_train)


In [ ]:
# Read new data from CSV file
new_data = pd.read_csv('new_data.csv')
new_price_data = new_data['Close'].values


In [ ]:
# Execute the strategy on new data
new_signals = generate_signals(new_price_data, ema_window_size, std_window_size)
new_positions = execute_strategy(model, new_signals)


In [ ]:
# Execute the strategy with risk management on new data
new_equity_curve = apply_risk_management(new_positions, new_price_data)

# Visualize the results
plt.plot(new_price_data, label='Price Data')
plt.scatter(np.where(new_positions == 1)[0], new_price_data[new_positions == 1], marker='^', color='g', label='Buy Signal')
plt.scatter(np.where(new_positions == -1)[0], new_price_data[new_positions == -1], marker='v', color='r', label='Sell Signal')
plt.legend()
plt.show()

# Visualize the equity curve with risk management
plt.plot(new_equity_curve, label='Equity Curve')
plt.legend()
plt.xlabel('Trade')
plt.ylabel('Capital')
plt.title('Equity Curve with Risk Management')
plt.show()